In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Проект e-learning: вариант 2

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (10 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (7 баллов) 

4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)

5. Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.  (15 баллов) 

6. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Описание подхода можно найти тут. (23 балла)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться курсом. Обосновать свой выбор ты можешь с помощью фактов сдачи экзаменов, распределения студентов и уникального идентификатора курса.

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен*

Файлы: 

**assessments.csv** — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).

- code_module — идентификационный код предмета.
- code_presentation — семестр (Идентификационный код).
- id_assessment — тест (Идентификационный номер ассессмента).
- assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
- date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
- weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

**courses.csv** — файл содержит список предметов по семестрам.

- code_module — предмет (идентификационный код).
- code_presentation — семестр (идентификационный код).
- module_presentation_length — продолжительность семестра в днях.

**studentAssessment.csv** — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.

- id_assessment — тест (идентификационный номер).
- id_student — идентификационный номер студента.
- date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.
- is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).
- score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

**studentRegistration.csv** — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.

- code_module — предмет (идентификационный код).
- code_presentation — семестр (идентификационный код)
- id_student — идентификационный номер студента.
- date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
- date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.


- B - осенний семестр
- J - весенний семестр

---
## 1. Выгрузка и изучение данных для анализа. Формулирование понятия "Курс".
На первом этапе загрузим данные из таблиц csv в датафреймы Пандас, посмотрим на содержание таблиц, и сформулируем, что считать курсом.

In [212]:
assessments         = pd.read_csv('./var_2/assessments.csv')
courses             = pd.read_csv('./var_2/courses.csv')
studentAssessment   = pd.read_csv('./var_2/studentAssessment.csv')
studentRegistration = pd.read_csv('./var_2/studentRegistration.csv')

In [213]:
assessments.head(2)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0


In [214]:
assessments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB


In [215]:
courses.head(2)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269


In [216]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes


In [217]:
studentAssessment.head(2)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0


In [218]:
studentAssessment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  int64  
 1   id_student      173912 non-null  int64  
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB


In [219]:
studentRegistration.head(1)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN


In [220]:
studentRegistration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          32593 non-null  object 
 1   code_presentation    32593 non-null  object 
 2   id_student           32593 non-null  int64  
 3   date_registration    32548 non-null  float64
 4   date_unregistration  10072 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


---
### Формулируем нужные для решения термины
Попробуем сформулировать, что считать курсом в рамках поставленных задач.
Судя по всему, данные - об обучении студентов в университете или колледже США. 
Об этом можно судить по обозначению семестров (B - осенний семестр, J - весенний семестр).
В системе образования, принятой в США, курс - это процесс изучения определенной предметной области в течение одного или нескольких семестров, причем курсы отличаются по сложности (сложность отражается в названии курса в виде числа, например, курс CS110 и курс CS254 - это курсы по computer science, но второй - более сложный).
Можно предположить, что курс - это изучение предмета в течение одного семестра, завершающееся экзаменом.

В таблице assessments мы видим поле code_module - идентификатор предмета.
Просмотр данной таблицы показал, что действительно, в рамках одного предмета проводится несколько видов аттестации, за которые выставляются оценки (CMA, TMA), а один раз в семестр проводится экзамен.
Продолжительность изучения каждого предмета - от 2 до 4 семестров.
В решении задач данного проекта будем пользоваться следующими формулировками:
- Курс - это изучение предмета в течение одного семестра, завершающееся экзаменом.
- Успешное прохождение курса: сдача экзамена курса с оценкой не менее 40 баллов.
- завершаемость курса - это кол-во успешных экзаменов / кол-во всех попыток сдать экзамен для всех студентов, предпринимавших попытки сдать экзамен.
- Срок сдачи курса: это раница между датой регистрации на курс (date_registration в датафрейме studentRegistration) и датой успешной сдачи экзамена по курсу (date_submitted в датафрейме studentAssessment). Если студент зарегистрировался на курс до начала его изучения (date_registration < 0), то принимаем дату старта обучения = 0.

In [221]:
# в сформированной сводной таблице подсчитано количество аттестаций в течение каждого из семестров
assessments.pivot_table(index='code_module', columns='code_presentation', values='assessment_type', aggfunc='count')

code_presentation,2013B,2013J,2014B,2014J
code_module,,,,
AAA,NaN,6.0,NaN,6.0
BBB,12.0,12.0,12.0,6.0
CCC,NaN,NaN,10.0,10.0
DDD,14.0,7.0,7.0,7.0
EEE,NaN,5.0,5.0,5.0
FFF,13.0,13.0,13.0,13.0
GGG,NaN,10.0,10.0,10.0


---

Не совсем понятно, как учитывать текущую успеваемость (т.е.оценки тестов TMA, CMA) в выводе об успешном завершении курса, но предположим, что его можно сделать только по оценкам за экзамены. К такому выводу подталкивает и информация из задания: метрика "Завершаемость" содержит только переменные, связанные с экзаменами.



---
## 2. Анализ успеваемости студентов по курсам
### 2.1. Определение количества студентов, которые успешно сдали только один курс

In [222]:
# создадим датафрейм, содержащий информацию из трех исходных датафреймов, 
# в нем будет содержаться информация только о попытках сдачи экзаменов
exam_assessments = studentAssessment \
    .merge(assessments, how='left', on='id_assessment') \
    .merge(studentRegistration[['id_student', 'code_module', 'code_presentation', 'date_registration', 'date_unregistration']] , how='left', on=['id_student', 'code_module', 'code_presentation']) \
    .query('assessment_type == "Exam"')
exam_assessments.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,date_registration,date_unregistration
52923,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0,-74.0,NaN
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0,-22.0,NaN
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0,-22.0,NaN
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0,-281.0,NaN
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0,-28.0,NaN


In [223]:
# Выберем из датафрейма только те записи, в которых количество баллов > 40 и с пустой датой отмены регистрации, 
# сгруппируем по id студентов и посмотрим, сколько у нас строк, в которых только одна успешная сдача экзамена
exam_assessments.query('score>40 and date_unregistration.isna()') \
    .groupby('id_student').agg({'assessment_type':'count'}).query('assessment_type == 1').shape[0]

3706

### Вывод по 2.1:
3706 студентов успешно сдали только один курс

---

### 2.2. Выявление самого сложного и самого простого экзамена (курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*).

Сначала необходимо рассчитать завершаемость курса.
Для этого сначала посчитаем общее количество попыток сдать курс и количество успешных сдач.
Для этого будем использовать сформированный на предыдущем шаге датафрейм `exam_assessments`

In [224]:
# добавим колонку is_success, в которой будет 1, если экзамен успешно сдан, и 0, если не сдан
exam_assessments['is_success'] = exam_assessments.score.apply(lambda x: 1 if x>40 else 0)

In [225]:
# сформируем новый датафрейм, который будет содержать код предмета, семестр, 
# и количество успешных сдач success_passes - причем только по тем записям, в которых не было отмены регистрации
complet_exams=exam_assessments \
    .query('date_unregistration.isna()') \
    .groupby(['code_module', 'code_presentation'], as_index=False) \
    .agg({'is_success':'sum'}) \
    .rename(columns={'is_success':'success_passes'})

In [226]:
# добавим в него колонку all_passes (общее количество попыток сдать), а также колонку copletion - 
# завершаемость курса в % (округлим это значение до 2 знаков после запятой)
complet_exams['all_passes']=exam_assessments \
    .groupby(['code_module', 'code_presentation'], as_index=False) \
    .agg({'is_success':'count'}).is_success
complet_exams['copletion']=round(100*complet_exams['success_passes']/complet_exams['all_passes'],2)

In [227]:
# посмотрим, что поолучилось
complet_exams.sort_values('copletion')

,code_module,code_presentation,success_passes,all_passes,copletion
2,DDD,2013B,478,602,79.40
1,CCC,2014J,995,1168,85.19
5,DDD,2014J,825,950,86.84
0,CCC,2014B,650,747,87.01
3,DDD,2013J,856,968,88.43
4,DDD,2014B,478,524,91.22


In [228]:
# оказывается, студенты сдавали экзаменационные тесты только по предметам CCC и DDD
# проверим это: посмотрим id экзаменов по предметам AAA, BBB, EEE, FFF, GGG и попробуем найти по этим id 
# записи в датафрейме studentAssessment
exam_ids = assessments.query('assessment_type == "Exam" and code_module in ["AAA", "BBB", "EEE", "FFF", "GGG"]').id_assessment
studentAssessment.query('id_assessment in @exam_ids')

,id_assessment,id_student,date_submitted,is_banked,score


Ничего не нашлось! Получается, по предметам AAA, BBB, EEE, FFF, GGG не было попыток сдачи экзамена.


### Вывод по п.2.2:
Самый сложный экзамен - по курсу DDD-2013B - завершаемость курса составила 79.40%, а самый простой экзамен по курсу DDD-2014B - завершаемость курса составила 91.22%

---

### 2.3. Определение среднего срока сдачи экзаменов


Для этого нам нужны данные по каждому факту успешной сдачи экзамена:
    - Код предмета;
    - Код семестра;
    - id оценки;
    - Дата регистрации на прохождение курса по предмету;
    - Дата успешной сдачи экзамена;
    
Оценим средний срок сдачи экзамена в разрезе по предметам и семестрам.

---
у нас уже есть датафрейм exam_assessments, который содержит информацию о попытках сдачи экзамена.
студенты регистрировались на курсы заранее, но обучение началось у всех одновременно. Поэтому добавим
колонку start_date - она будет равна 0, если дата регистрации на курс отрицательная, и будет равна 
дате регистрации, если эта дата не менее 0. 

В данных о попытках сдачи экзаменов есть такие записи: успешные (>40 б.) попытки, но студенты затем отменяли регистрацию на курс. Получается, такие студенты сдавали экзамен, значит, мы знаем о сроках сдачи. Поэтому оставим в датафрейме данные о таких студентах, несмотря на то, что они покинули курс.
    

In [229]:
exam_assessments['start_date'] = exam_assessments.date_registration.apply(lambda x: 0 if x<0 else x)

In [230]:
# посмотрим, что получилось, а заодно проверим, есть ли регистрации на курс после начала семестра
exam_assessments[exam_assessments.start_date>0].head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,date_registration,date_unregistration,is_success,start_date
53267,24290,634816,234,0,82.0,CCC,2014B,Exam,NaN,100.0,5.0,NaN,1,5.0
53365,24290,2665399,230,0,96.0,CCC,2014B,Exam,NaN,100.0,6.0,NaN,1,6.0
53524,24290,440905,230,0,34.0,CCC,2014B,Exam,NaN,100.0,12.0,NaN,0,12.0
64085,24299,589463,244,0,82.0,CCC,2014J,Exam,NaN,100.0,23.0,NaN,1,23.0
64575,24299,684111,244,0,42.0,CCC,2014J,Exam,NaN,100.0,5.0,NaN,1,5.0


In [231]:
# Добавим еще одну колонку - в ней будет срок сдачи экзамена, 
# определенный как разность между датой сдачи и датой начала обучения
exam_assessments['pass_time'] = exam_assessments['date_submitted'] - exam_assessments['start_date']
exam_assessments.head(1)

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,date_registration,date_unregistration,is_success,start_date,pass_time
52923,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0,-74.0,NaN,0,0.0,230.0


In [232]:
# Теперь можно выбрать только те попытки, которые были успешными, 
# сгруппировать по предметам и посчитать средний срок сдачи, округлив его затем до целого.
exam_assessments.query('is_success==1') \
    .groupby('code_module', as_index=False) \
    .agg({'pass_time':'mean'}).round()

,code_module,pass_time
0,CCC,239.0
1,DDD,238.0


In [233]:
# можно также посмотреть средние сроки сдачи экзаменов в разрезе не только предметов, но и семестров
exam_assessments.query('is_success==1') \
    .groupby(['code_module', 'code_presentation'], as_index=False) \
    .agg({'pass_time':'mean'}).round() \
    .sort_values('pass_time')

,code_module,code_presentation,pass_time
2,DDD,2013B,230.0
0,CCC,2014B,232.0
4,DDD,2014B,235.0
3,DDD,2013J,239.0
5,DDD,2014J,243.0
1,CCC,2014J,244.0


### Вывод по 2.3:
Средние сроки сдачи экзаменов по предметам CCC и DDD почти одинаковые: 239 и 238 дней соответственно.

Если рассмотреть данные по курсам, то быстрее всего в реднем студенты сдавали экзамен по курсу DDD-2013B (230 дней), а дольше всего - по курсу CCC-2014J (244 дня).

---

### 2.4. Выявление самых популярных (ТОП-3) предметов, а также предметов с самым большим оттоком (ТОП-3)


In [234]:
# посмотрим на ТОП-3 самых популярных предметов:
studentAssessment \
    .merge(assessments, how='left', on='id_assessment') \
    .merge(studentRegistration, how='left', on=['id_student', 'code_module', 'code_presentation']) \
    .groupby('code_module') \
    .agg({'date_registration':'count'}) \
    .rename(columns={'date_registration':'registrations_number'}) \
    .sort_values('registrations_number', ascending=False) \
    .head(3)

,registrations_number
code_module,
FFF,54815
BBB,43032
DDD,30859


In [235]:
# Теперь посмотрим на предметы с самым большим оттоком:
studentAssessment \
    .merge(assessments, how='left', on='id_assessment') \
    .merge(studentRegistration, how='left', on=['id_student', 'code_module', 'code_presentation']) \
    .query('date_unregistration.notna()') \
    .groupby('code_module') \
    .agg({'date_unregistration':'count'}) \
    .rename(columns={'date_unregistration':'unregistrations_number'}) \
    .sort_values('unregistrations_number', ascending=False) \
    .head(3)

,unregistrations_number
code_module,
DDD,3523
FFF,3397
BBB,2693


### Вывод по п.2.4:
Самый популярный предмет - это  FFF, количество регистраций составило 54815.
Больше всего отмен регистраций было с предмета DDD - их количество составило 3523.

---

### 2.5. Выявление семестров с самой низкой завершаемостью курсов и самыми долгими сроками сдачи курсов

На предыдущих этапах мы рассмотрели распределение завершаемости курсов и рассчитали средний срок сдачи курсов по семестрам. Поскольку мы сформулировали, что критерий завершения курса - это успешная сдача экзамена за семестр, получается, что будем рассматривать только предметы CCC  и DDD. 

In [238]:
# у нас уже сформирован датафрейм, содержащий колонку с завершаемостью курса
complet_exams

,code_module,code_presentation,success_passes,all_passes,copletion
0,CCC,2014B,650,747,87.01
1,CCC,2014J,995,1168,85.19
2,DDD,2013B,478,602,79.40
3,DDD,2013J,856,968,88.43
4,DDD,2014B,478,524,91.22
5,DDD,2014J,825,950,86.84


In [239]:
# сохраним данные о среднем сроке сдачи курсов в датафрейм passtime_df
passtime_df = exam_assessments.query('is_success==1') \
    .groupby(['code_module', 'code_presentation'], as_index=False) \
    .agg({'pass_time':'mean'}).round() \
    .sort_values('pass_time')

In [244]:
# Посмотрим, как распределяется завершаемость курса и средний срок сдачи по семестрам:
complet_exams \
    .merge(passtime_df, how='left', on=['code_module', 'code_presentation']) \
    .groupby('code_presentation') \
    .agg({'copletion':'mean', 'pass_time':'mean'}) \
    .sort_values('copletion')

,copletion,pass_time
code_presentation,,
2013B,79.400,230.0
2014J,86.015,243.5
2013J,88.430,239.0
2014B,89.115,233.5


### Вывод по п.2.5:

Семестр с самой низкой завершаемостью курса - это 2013B, а семестр с самым большим средним сроком сдачи курсов - это 2014J

---

## 3. RFM-сегментация
---
Выберем следующие метрики:
    
    - R - среднее время сдачи одного экзамена    
    - F - завершаемость курсов
    - M - среднее количество баллов за экзамен
    
Для RFM-анализа в качестве исходного датафрейма будем использовать датафрейм `exam_assessments`

In [245]:
exam_assessments.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,date_registration,date_unregistration,is_success,start_date,pass_time
52923,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0,-74.0,NaN,0,0.0,230.0
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0,-22.0,NaN,1,0.0,234.0
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0,-22.0,NaN,1,0.0,230.0
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0,-281.0,NaN,1,0.0,230.0
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0,-28.0,NaN,1,0.0,234.0


In [274]:
# Выберем из него только нужные данные и сохраним в новый датафрейм:
rfm_df = exam_assessments \
    .query('date_unregistration.isna()') \
    .groupby('id_student', as_index=False) \
    .agg({'pass_time':'mean', 'is_success':'count', 'score':'mean'}) \
    .rename(columns={'is_success':'all_passes'})


In [275]:
# добавим данные по завершаемости курса, 
# разделив количество успешных сдач на общее количество сдач и выразив в процентах:
rfm_df['completion'] = 100*exam_assessments \
    .query('date_unregistration.isna()') \
    .groupby('id_student', as_index=False) \
    .agg({'is_success':'sum'}).is_success / rfm_df['all_passes']
# после этого удалим уже ненужную нам колонку all_passes:
rfm_df=rfm_df.drop('all_passes', axis=1)
rfm_df.shape

(4632, 4)

In [269]:
rfm_df.head()

,id_student,pass_time,score,completion
0,23698,243.0,80.0,100.0
1,24213,236.0,58.0,100.0
2,27116,243.0,96.0,100.0
3,28046,237.0,40.0,0.0
4,28787,243.0,44.0,100.0


#### В ситуациях, когда речь идет о продажах, R - это давность заказа, F - частота покупок, M - средний чек. В нашем случае мы адаптируем этот подход к данным об обучении.
Поэтому предлагаем оценить три критерия сегментации таким образом - оценим студентов по каждому сегменту по 3-балльной шкале: 

**R(средний срок сдачи)** : 1 - долго сдающие; 2 - студенты со средним сроком сдачи экзаменов; 3 - быстро сдающие.

**F(завершаемость курса)** : 1 - с низкой завершаемостью; 2 - со средней завершаемостью; 3 - с высокой завершаемостью.

**M(среднее количество баллов)** : 1 - низкий балл; 2 - средний балл; 3 - с высоким количеством баллов.

Затем мы можем ранжировать студентов по этим трем критериям (например, получившие RFM="111" студенты явно попадают в группу риска - они затягивают сроки сдачи экзамена, не всегда завершают курс, получают низкие баллы за экзамены).

---

Но для оценивания нужно сначала выбрать диапазоны сегментирования.

In [277]:
# посмотрим, как варьируются значения наших критериев
rfm_df.describe()

,id_student,pass_time,score,completion
count,4.632000e+03,4632.000000,4632.000000,4632.000000
mean,7.257552e+05,238.381045,65.124892,85.870035
std,5.753950e+05,6.063214,20.467329,34.580166
min,2.369800e+04,136.000000,0.000000,0.000000
25%,5.014048e+05,234.000000,50.000000,100.000000
50%,5.884875e+05,241.000000,66.000000,100.000000
75%,6.463530e+05,243.000000,82.000000,100.000000
max,2.698251e+06,285.000000,100.000000,100.000000


В целом для разделения на диапазоны можно использовать простой подход: равномерно разделить разброс значений (max-min) на три равных части и таким образом посчитать границы диапазонов. Причем для F и M эти границы будут одинаковы, так как равны их максимумы (100) и минимумы (0).


In [279]:
# посмотрим на всякий случай, какие значения принимает критерий completion, а то у него все квартили одинаковы
rfm_df.completion.unique()

array([100.,   0.,  50.])

In [292]:
# подготовим функции для оценивания студентов по криитериям:
def r_seg(x):
    if x < 170:
        return 1
    elif 170<=x<207:
        return 2
    elif x>=207:
        return 3    

In [293]:
def f_m_seg(x):
    if x < 33:
        return 1
    elif 33<=x<66:
        return 2
    elif x>=66:
        return 3    

In [297]:
rfm_df['R'] = rfm_df['pass_time'].apply(lambda x: r_seg(x))
rfm_df['F'] = rfm_df['completion'].apply(lambda x: f_m_seg(x))
rfm_df['M'] = rfm_df['score'].apply(lambda x: f_m_seg(x))

In [301]:
rfm_df.sort_values('R')

,id_student,pass_time,score,completion,R,F,M
79,84605,154.0,76.0,100.0,1,3,3
422,305535,136.0,62.0,100.0,1,3,2
934,467413,182.0,42.0,100.0,2,3,2
0,23698,243.0,80.0,100.0,3,3,3
3091,627295,243.0,49.0,100.0,3,3,2
...,...,...,...,...,...,...,...
1542,544587,234.0,33.0,0.0,3,1,2
1541,544576,242.0,62.0,100.0,3,3,2
1540,544490,236.0,46.0,100.0,3,3,2
1592,548473,233.0,36.0,0.0,3,1,2


В итоге мы получаем возможность разделить всех студентов на 27 кластеров (111, 112, 113, 121, 122, 123 и т.д).
Попробуем оценить количество студентов в каждом кластере от 111 до 333:

In [305]:
rfm_df.groupby(['R', 'F', 'M'], as_index=False).agg({'id_student':'count'})

,R,F,M,id_student
0,1,3,2,1
1,1,3,3,1
2,2,3,2,1
3,3,1,1,277
4,3,1,2,361
5,3,2,2,33
6,3,3,2,1625
7,3,3,3,2333


### Вывод по п.3:
Из 27 возможных кластеров у нас есть только 8.
Большая часть студентов (2333 чел.) находятся в кластере 333, то есть они быстро сдают экзамены, имеют высокую завершаемость курсов и показывают высокий результат (не менее 66 б.) в среднем за экзамен.

Если оценивать угрозу отчисления/повторного прохождения курсов, то стоит обратить внимание на кластеры с F=1, так как именно они делают больше попыток до успешной сдачи.

Если оценивать качество обучения, то в фокусе кластеры с M=1 (строка с индексом 3, 277 чел.): они успевают сдавать экзамены вовремя, но у них не только низкая завершаемость курса, но и средний балл за экзамены меньше 33.